# TODO:
- Include more info from solver (maxiter, opts, iter, tol, etc.)

# Imports

In [ ]:
# stdlib
import sqlite3
from datetime import datetime
import os

# 3rd party
import numpy as np
import pandas as pd
import netCDF4 as nc
import tempfile

In [ ]:
# Just for testing
import shutil
from IPython.display import display

# Functions

In [ ]:
def create_table(run_name, prefix='.'):
    """
    Create sqlite db file, create table, and return connection.
    Assume run_name is unique (e.g. includes date.)
    """
    
    create_table_template = '''
    CREATE TABLE {db_name} (

        /* Inputs */
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        a_kelp REAL,
        a_water REAL,
        b REAL,
        ns REAL,
        nz REAL,
        na REAL,
        num_dens REAL,
        kelp_dist CHAR(32),
        fs REAL,
        fr REAL,
        ft REAL,
        max_length REAL,
        length_std REAL,
        zmax REAL,
        rope_spacing REAL,
        date CHAR(64),

        /* Outputs (NETCDF) 
        Contains:
          - radiance (ns, ns, nz, na, na) 
          - irradiance (ns, ns, nz)
          - p_kelp (ns, ns, nz)
          - avg_irrad (nz)
          - perc_irrad (nz)
        */
        results_path CHAR(256)
        )
    '''
    
    base_dir = os.path.join(prefix, run_name)
    db_file = os.path.join(base_dir, run_name+'.db')
    # Will fail if `run_name` is not unique in `prefix`
    os.mkdir(base_dir)
    
    conn = sqlite3.connect(db_file)
    conn.execute(create_table_template.format(db_name=run_name))
    
    return conn

In [ ]:
def insert_run(conn, **params):
    insert_template = '''
    INSERT INTO {db_name} VALUES (
        NULL, /* id (autoincrement) */
        {a_kelp},
        {a_water},
        {b},
        {ns},
        {nz},
        {na},
        {num_dens},
        '{kelp_dist}',
        {fs},
        {fr},
        {ft},
        {max_length},
        {length_std},
        {zmax},
        {rope_spacing},
        '{date}',
        '{results_path}'
        )
    '''
    
    return conn.execute(insert_template.format(**params))

In [ ]:
def create_nc(results_path, **results):
    # Create file
    rootgrp = nc.Dataset(results_path, 'w', format='NETCDF4')
    
    # Get dimension sizes
    nx = len(results['x'])
    ny = len(results['y'])
    nz = len(results['z'])
    ntheta = len(results['theta'])
    nphi = len(results['phi'])
    
    # Create Dimensions
    x = rootgrp.createDimension('x', nx)
    y = rootgrp.createDimension('y', ny)
    z = rootgrp.createDimension('z', nz)
    theta = rootgrp.createDimension('theta', ntheta)
    phi = rootgrp.createDimension('phi', nphi)
    
    # Create dimension variables
    xvals = rootgrp.createVariable('x','f8', ('x',))
    yvals = rootgrp.createVariable('y','f8', ('y',))
    zvals = rootgrp.createVariable('z','f8', ('z',))
    thetavals = rootgrp.createVariable('theta','f8', ('theta',))
    phivals = rootgrp.createVariable('phi','f8', ('phi',))
    
    # Assign dimension variables
    xvals[:] = results['x']
    yvals[:] = results['y']
    zvals[:] = results['z']
    thetavals[:] = results['theta']
    phivals[:] = results['phi']
    
    # Create results variables
    rad = rootgrp.createVariable('rad','f8', ('x', 'y', 'z', 'theta', 'phi'))
    irrad = rootgrp.createVariable('irrad','f8', ('x', 'y', 'z'))
    avg_irrad = rootgrp.createVariable('avg_irrad','f8', ('z',))
    perc_irrad = rootgrp.createVariable('perc_irrad','f8', ('z',))

    # Assign results variables
    rad[:] = results['rad']
    irrad[:] = results['irrad']
    avg_irrad[:] = results['avg_irrad']
    perc_irrad[:] = results['perc_irrad']
    
    # Close file
    rootgrp.close()

In [ ]:
def get_random_unused_filename(dir='.', prefix='', suffix=''):
    with tempfile.NamedTemporaryFile(dir=dir, prefix=prefix, suffix=suffix) as fh:
        filename = fh.name
    return filename

# Test Functions

In [ ]:
def create_phony_kelp(results_path):
    ns, nz, na = np.random.randint(10, 20, size=3)
    # Make sure na is even
    if na % 2 == 1:
        na -= 1
        
    a_kelp, a_water, b = np.random.rand(3)
    num_dens = np.random.rand()
    fs, fr, ft = np.random.rand(3)
    kelp_dist = 'top-heavy'
    max_length, length_std = np.random.rand(2)
    zmax, rope_spacing = 10 * np.random.rand(2)
    date = datetime.now().ctime()
    
    return {
        'a_kelp': a_kelp,
        'a_water': a_water,
        'b': b,
        'ns': ns,
        'nz': nz,
        'na': na,
        'num_dens': num_dens,
        'kelp_dist': kelp_dist,
        'fs': fs,
        'fr': fr,
        'ft': ft,
        'max_length': max_length,
        'length_std': length_std,
        'zmax': zmax,
        'rope_spacing': rope_spacing,
        'date': date,
        'results_path': results_path
    }

In [ ]:
def create_phony_results(ns, nz, na):
    results = {
        # Dimension values
        # These are grid-starts, not grid-centers, but it's phony anyhow.
        'x': np.linspace(0, 1, ns),
        'y': np.linspace(0, 1, ns),
        'z': np.linspace(0, 1, nz),
        'theta': np.linspace(0, 2*np.pi, na),
        'phi': np.linspace(0, np.pi, na),
        
        # Results
        'rad': np.random.rand(ns, ns, nz, na, na),
        'irrad': np.random.rand(ns, ns, nz),
        'avg_irrad': np.random.rand(nz),
        'perc_irrad': np.random.rand(nz)
    }
    
    return results

In [ ]:
def run_phony(conn, run_name, base_dir):
    results_path = get_random_unused_filename(dir=base_dir, suffix='.nc')
    
    params = create_phony_kelp(results_path)
    ns = params['ns']
    nz = params['nz']
    na = params['na']
    results = create_phony_results(ns, nz, na)
    
    create_nc(results_path, **results)
    insert_run(conn, db_name=run_name, **params)

def run_phony_set(run_name, prefix='.', num_entries=10):
    base_dir = os.path.join(prefix, run_name)
    conn = create_table(run_name, prefix)
    for i in range(num_entries):
        run_phony(conn, run_name, base_dir)
        
    return conn

In [ ]:
def table_to_df(conn, table_name):
    select_cmd = 'SELECT * FROM {table_name}'.format(table_name=table_name)
    cursor = conn.execute(select_cmd)
    columns = [col[0] for col in cursor.description]
    data = [x for x in cursor]
    
    df = pd.DataFrame(data, columns=columns)
    return df

In [ ]:
def query_results(conn, table_name, **kwargs):
    # Sanitize string values
    for key, val in kwargs.items():
        if isinstance(val, str):
            kwargs[key] = '"{}"'.format(val)
    
    # Form SQL WHERE clause
    where_clause = ' AND '.join([
        '{}={}'.format(key, value)
        for key, value in kwargs.items()
    ])
    
    # Form entire query
    query = '''SELECT results_path FROM {table_name}
    WHERE {where_clause}
    '''.format(
        table_name=table_name,
        where_clause=where_clause
    )
    
    # Execute query
    cursor = conn.execute(query)
    results_list = [x for x in cursor]
    
    # Extract matching datasets
    datasets = [nc.Dataset(results[-1]) for results in results_list]
    return datasets

# Test

In [ ]:
run_name = 'test1'
prefix='.'
base_dir = os.path.join(prefix, run_name)

# Remove dir if it exists
shutil.rmtree(base_dir)

# Run set
conn = run_phony_set(run_name, prefix=prefix, num_entries=10)
display(table_to_df(conn, run_name))

# Look at results

In [ ]:
ds, *_ = query_results(conn, 'test1', kelp_dist='top-heavy', id=2)

In [ ]:
ds['z']

In [ ]:
ds['z'][:]

In [ ]:
ds['perc_irrad']

In [ ]:
ds['perc_irrad'][:]